# TinyVGG for Fasnion MNIST dataset

In [1]:
import torch, torchvision
from torch import nn

In [2]:
transform_simple = torchvision.transforms.Compose([
    torchvision.transforms.Resize(size=(64, 64)),
    torchvision.transforms.ToTensor()
])

train_dataset = torchvision.datasets.FashionMNIST(
    root=".datasets",
    train=True,
    transform=transform_simple,
    target_transform=None,
    download=True
)

test_dataset = torchvision.datasets.FashionMNIST(
    root=".datasets",
    train=False,
    transform=transform_simple,
    target_transform=None,
    download=True
)

In [3]:
import torch.utils.data

train_dataloader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=32,
    shuffle=True
)

test_dataloader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=32,
    shuffle=False
)

In [4]:
len(train_dataloader), len(test_dataloader)

(1875, 313)

In [5]:
class FashionMNISTModelV0(nn.Module):
    def __init__(self, in_shape, hidden_units, out_shape):
        super().__init__()
        self.conv_layer_1 = nn.Sequential(
            nn.Conv2d(in_channels=in_shape, out_channels=hidden_units, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3)
        )
        self.conv_layer_2 = nn.Sequential(
            nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_units*5*5, out_features=out_shape)
        )
    
    def forward(self, X):
        return self.classifier.forward(
            self.conv_layer_2.forward(
                self.conv_layer_1.forward(X)
            )
        )

In [6]:
model = FashionMNISTModelV0(
    in_shape=1,
    hidden_units=40,
    out_shape=len(train_dataset.classes)
)

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.0005)

In [8]:
epochs = 3

for epoch in range(epochs):
    for batch, (X, y) in enumerate(train_dataloader):
        model.train()
        y_logits = model.forward(X)
        loss = loss_fn(y_logits, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(f"Batch: {batch} | Loss: {loss}", end="\r")

    loss = 0
    for batch, (X, y) in enumerate(test_dataloader):
        model.eval()
        with torch.inference_mode():
            y_logits = model.forward(X)
            loss += loss_fn(y_logits, y).item()
        print(f"Batch: {batch} | Loss: {loss / (batch+1)}", end="\r")
    loss = loss / (batch+1)
    
    print(f"Epoch: {epoch} | Loss: {loss}")

Epoch: 0 | Loss: 0.38661234802045763633
Epoch: 1 | Loss: 0.37190785585120084845
Epoch: 2 | Loss: 0.309631704355771535334


In [9]:
y_preds = []
y_true = []

for batch, (X,  y) in enumerate(test_dataloader):
    model.eval()
    with torch.inference_mode():
        y_logits = model.forward(X)
        y_pred = y_logits.argmax(dim=1)
        y_preds = y_preds + y_pred.tolist()
        y_true = y_true + y.tolist()

In [10]:
from torchmetrics import Accuracy

accuracy = Accuracy(task="multiclass", num_classes=len(train_dataset.classes))
accuracy = accuracy(torch.Tensor(y_preds), torch.Tensor(y_true))

print(f"Accuracy: {accuracy*100:.2f}%")

Accuracy: 89.23%
